In [2]:
import pandas as pd
import geopandas as gpd
from osgeo import gdal

In [17]:
my_data = pd.read_pickle("/Users/will/dev/morel-model/data/data_zcta_20230423_prism.pkl")

In [18]:
their_data = pd.read_csv("/Users/will/dev/morel-model/data/weather/2023/202304_zcta_weather_daily.csv")

In [19]:
their_data['zcta'] = their_data['zipcode'].astype(str).str.zfill(5)

In [20]:
their_data_subset = their_data[their_data['date'] == 20230423]

In [21]:
their_data_subset

,st_abb,st_code,county_name,fips,zipcode,stability,date,ppt,tmax,tmin,...,dday_a29C,dday_a30C,dday_a31C,dday_a32C,dday_a33C,dday_a34C,dday_b15C,dday_b18C,dday_b21C,zcta
22,MA,25,Hampden,25013,1001,stable,20230423,33.588,16.710,9.885,...,0.0,0.0,0.0,0.0,0.0,0.0,2.076,4.703,7.703,01001
52,MA,25,Hampshire,25015,1002,stable,20230423,25.485,15.002,7.158,...,0.0,0.0,0.0,0.0,0.0,0.0,3.920,6.920,9.920,01002
82,MA,25,Hampshire,25015,1003,stable,20230423,25.775,16.155,8.246,...,0.0,0.0,0.0,0.0,0.0,0.0,2.989,5.799,8.799,01003
112,MA,25,Worcester,25027,1005,stable,20230423,27.465,13.037,6.786,...,0.0,0.0,0.0,0.0,0.0,0.0,5.089,8.089,11.089,01005
142,MA,25,Hampshire,25015,1007,stable,20230423,21.900,15.146,7.099,...,0.0,0.0,0.0,0.0,0.0,0.0,3.886,6.878,9.878,01007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975142,WA,53,Walla Walla,53071,99363,stable,20230423,0.330,18.468,6.351,...,0.0,0.0,0.0,0.0,0.0,0.0,3.403,5.630,8.591,99363
975172,WA,53,Adams,53001,99371,stable,20230423,0.000,16.302,0.779,...,0.0,0.0,0.0,0.0,0.0,0.0,6.620,9.459,12.459,99371
975202,WA,53,Asotin,53003,99401,stable,20230423,0.583,12.849,0.718,...,0.0,0.0,0.0,0.0,0.0,0.0,8.216,11.216,14.216,99401
975232,WA,53,Asotin,53003,99402,stable,20230423,0.533,13.448,1.527,...,0.0,0.0,0.0,0.0,0.0,0.0,7.512,10.512,13.512,99402


In [25]:
my_data['zcta'] = my_data['ZCTA5CE10'].astype(str).str.zfill(5)

In [26]:
joined_data = pd.merge(my_data[['zcta', 'tmax_avg']], their_data_subset[['zcta', 'tmax']], how='inner', on=['zcta'])

In [27]:
len(their_data_subset)

32509

In [28]:
len(my_data)

33144

In [29]:
len(joined_data)

32509

In [30]:
joined_data['diff'] = joined_data['tmax'] - joined_data['tmax_avg']
joined_data['abs_diff'] = abs(joined_data['tmax'] - joined_data['tmax_avg'])

In [31]:
joined_data.describe()

,tmax_avg,tmax,diff,abs_diff
count,32509.000000,32509.000000,32509.000000,32509.000000
mean,16.032930,16.045514,0.012584,0.072181
std,7.838313,7.843216,0.138457,0.118822
min,-3.847071,-4.096000,-2.384416,0.000000
25%,9.715134,9.734000,-0.029209,0.012873
50%,15.866986,15.900000,0.000865,0.034596
75%,22.463531,22.475000,0.040572,0.081534
max,36.399422,36.424000,2.227470,2.384416


In [58]:
joined_data.sort_values(['diff'])[:40]

,zcta,tmax_avg,tmax,diff
25563,87514,8.895416,6.511,-2.384416
11968,95010,18.025064,15.801,-2.224064
17623,08740,19.950199,18.278,-1.672199
30373,92382,22.390200,20.784,-1.606200
29795,92325,23.663782,22.071,-1.592782
8454,97118,16.106249,14.561,-1.545249
29366,91739,27.244848,25.703,-1.541848
27950,94037,19.261326,17.722,-1.539326
29817,86331,21.646507,20.231,-1.415507
29728,93222,17.391201,15.983,-1.408201


In [60]:
their_data[(their_data['zcta'] == '87514') & (their_data['date'] == 20230423)]

,st_abb,st_code,county_name,fips,zipcode,stability,date,ppt,tmax,tmin,...,dday_a29C,dday_a30C,dday_a31C,dday_a32C,dday_a33C,dday_a34C,dday_b15C,dday_b18C,dday_b21C,zcta
879682,NM,35,Taos,35055,87514,stable,20230423,0.014,6.511,-5.003,...,0.0,0.0,0.0,0.0,0.0,0.0,14.246,17.246,20.246,87514


In [61]:
my_data[(my_data['zcta'] == '87514')]

,ZCTA5CE10,tmax_avg,zcta
26072,87514,8.895416,87514


In [64]:
gpd.read_file('~/Downloads/PRISM_tmin_stable_4kmM3_200904_bil/PRISM_tmin_stable_4kmM3_200904_bil.bil')

DriverError: '/Users/will/Downloads/PRISM_tmin_stable_4kmM3_200904_bil/PRISM_tmin_stable_4kmM3_200904_bil.bil' not recognized as a supported file format.

In [3]:
def ReadBilFile(bil):
    gdal.GetDriverByName('EHdr').Register()
    img = gdal.Open(bil)
    band = img.GetRasterBand(1)
    data = band.ReadAsArray()
    return data

In [10]:
x = ReadBilFile('/Users/Will/Downloads/PRISM_tmin_stable_4kmM3_200904_bil/PRISM_tmin_stable_4kmM3_200904_bil.bil')

In [15]:
x.size

872505